# AutoGen

In this code sample, you will use the [**AutoGen**](https://aka.ms/ai-agents/autogen) **AI** framework to create a basic agent.
The goal of this sample is to show you the steps that we will later use in the additional code samples when implementing the different agentic patterns.

## Prepare the project

Import necessary packages:

In [1]:
import os

Import necessary entities:

In [2]:
from typing import Any
from dotenv import load_dotenv
from IPython.display import HTML, display
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.agents import AssistantAgent
from azure.core.credentials import AzureKeyCredential
from autogen_core.models import UserMessage, CreateResult
from autogen_ext.models.azure import AzureAIChatCompletionClient

Load enviroment for using a system variables:

In [3]:
load_dotenv();

## Work with a client

In this sample, we will use [**GitHub** models](https://aka.ms/ai-agents-beginners/github-models) for access to the **LLM**.
The `model` is defined as `gpt-4o-mini`.
Try changing the model to another model available on the **GitHub** models marketplace to see the different results.
As a quick test, we will just run a simple prompt - `What is the capital of France`.

Create the client:

In [4]:
client: AzureAIChatCompletionClient = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN", ), ),
    model_info={
        "vision": True,
        "json_output": True,
        "family": "unknown",
        "function_calling": True,
    },
)

Make a request to **LLM**:

In [5]:
request_result: CreateResult = await client.create([UserMessage(
    content="What is the capital of France?",
    source="user",
), ], )

Check request result:

In [6]:
print(request_result, )

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=14, completion_tokens=8) cached=False logprobs=None thought=None


## Define a agent

Now that we have set up the `client` and confirmed that it is working, let us create an `AssistantAgent`.
Each agent can be assigned a:
`name` - A short hand name that will be useful in referencing it in multi-agent flows.
`model_client` - The client that you created in the earlier step.
`tools` - Available tools that the agent can use to complete a task.
`system_message` - The metaprompt that defines the task, behavior and tone of the **LLM**.
You can change the system message to see how the **LLM** responds.

In [7]:
agent: AssistantAgent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations.",
)

## Run the agent

The below function will run the agent.
We use the the `on_message` method to update the agent's state with the new message.
In this case, we update the state with a new message from the user which is `"Plan me a great sunny vacation"`.
You can change the message content to see how the **LLM** responds differently.

In [8]:
async def assistant_run() -> None:
    """
    Executes a conversational interaction with an AI assistant, formats the
    exchange in HTML, and displays the result in an interactive environment.
    """

    user_query: str = "Plan me a great sunny vacation."
    html_output: str = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"
    request_response: Any = await agent.on_messages(
        [TextMessage(
            content=user_query,
            source="user",
        ), ],
        cancellation_token=CancellationToken(),
    )
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += (
        "<div style='margin-left:20px; white-space:pre-wrap'>" +
        f"{request_response.chat_message.content}</div>"
    )
    html_output += "</div>"

    display(HTML(html_output, ), )

Launch the agent workflow:

In [9]:
await assistant_run()